# QC from mzML runs 

In this notebook, we will explore the quality control (QC) of mass spectrometry data stored in mzML format. The goal is to ensure that the data is suitable for further analysis and to identify any potential issues that may affect the results. This code is specifically designed to work with mzML files from SCAPIS DIA experiements with Khue. There are around 16 plates of plasma samples, each plate containing 96 samples. The mzML files are stored in a specific directory structure, and we will process them to extract relevant information for QC.


In [ ]:
# Import packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymzml


In [ ]:
# Path to mzML 
mzml_path = 'mzML/plate1_mzML'

In [ ]:
# Read mzML filesdef read_mzml_files(directory):
mzml_files = [f for f in os.listdir(mzml_path) if f.endswith('.mzML')]
data = []
for file in mzml_files:
    file_path = os.path.join(mzml_path, file)
    with mzml.MzML(file_path) as reader:
        for spectrum in reader:
            if 'm/z array' in spectrum and 'intensity array' in spectrum:
                mz = spectrum['m/z array']
                intensity = spectrum['intensity array']
                data.append({
                    'file': file,
                    'mz': mz,
                    'intensity': intensity,
                    'scan_time': spectrum.get('scan start time', None)
                })
# pd.DataFrame(data)


In [ ]:
data

In [ ]:
# Write a simpple script that read mzML fiel with pymzml and extract the m/z and intensity values, and scan time.
def read_mzml_with_pymzml(file_path):
    run = pymzml.run.Reader(file_path)
    all_data = []
    
    # print total number of spectra
    print(f'Total number of spectra: {len(run)}')
    
    for spectrum in run:
        if spectrum.ms_level == 1:  # Only process MS1 spectra
            mz_array = spectrum.mz
            intensity_array = spectrum.i
            scan_time = spectrum.scan_time
            
            # Create individual records for each m/z, intensity pair
            for mz_val, int_val in zip(mz_array, intensity_array):
                all_data.append({
                    'mz': mz_val,
                    'intensity': int_val,
                    'scan_time': scan_time
                })
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort by scan time
    df.sort_values(by='scan_time', inplace=True)
    df.reset_index(drop=True, inplace=True)
    # convert scan_time to float 
    # df['scan_time'] = df['scan_time'].astype(float) 

    return df

In [ ]:
data_df = read_mzml_with_pymzml("mzML/plate1_mzML/20241127_Scapis_DIA_P1A1.mzML")
# Sort the data by scan time
# data_df.sort_values(by='scan_time', inplace=True)

In [ ]:
data_df.tail(10)

In [ ]:
def print_max_spec_id(mzml_file):
    """
    Print the maximum spectrum ID from an mzML file.
    
    Parameters:
    mzml_file (str): Path to the mzML file
    """
    run = pymzml.run.Reader(mzml_file)
    max_spec_id = 0
    
    for spec in run:
        # Extract numeric part of spec.ID if it's a string
        try:
            if isinstance(spec.ID, str):
                # Extract numbers from the ID string
                spec_id_num = int(''.join(filter(str.isdigit, spec.ID)))
            else:
                spec_id_num = int(spec.ID)
            
            max_spec_id = max(max_spec_id, spec_id_num)
            
        except ValueError:
            # Skip if ID cannot be converted to integer
            continue
    
    print(f"Maximum spectrum ID: {max_spec_id}")
    return max_spec_id

# Usage
mzml_file = "mzML/plate1_mzML/20241127_Scapis_DIA_P1A1.mzML"
print_max_spec_id(mzml_file)

In [ ]:
mzml_file = "mzML/plate1_mzML/20241127_Scapis_DIA_P1A1.mzML"
run = pymzml.run.Reader(mzml_file)
for n, spec in enumerate(run):
    print(
        "Spectrum {0}, MS level {ms_level} @ RT {scan_time:1.5f}".format(
            spec.ID, ms_level=spec.ms_level, scan_time=spec.scan_time_in_minutes()
        )
    )
print("Parsed {0} spectra from file {1}".format(n, mzml_file))
print()

In [ ]:
def plot_spectrum_vs_scantime(mzml_file):
    """
    Plot a dot plot of spectrum ID vs scan time from an mzML file.
    
    Parameters:
    mzml_file (str): Path to the mzML file
    """
    run = pymzml.run.Reader(mzml_file)
    
    spectrum_ids = []
    scan_times = []
    
    for spec in run:
        try:
            # Extract numeric part of spec.ID if it's a string
            if isinstance(spec.ID, str):
                spec_id_num = int(''.join(filter(str.isdigit, spec.ID)))
            else:
                spec_id_num = int(spec.ID)
            
            spectrum_ids.append(spec_id_num)
            scan_times.append(spec.scan_time_in_minutes())
            
        except (ValueError, TypeError):
            # Skip if ID cannot be converted to integer or scan_time is None
            continue
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.scatter(scan_times, spectrum_ids, alpha=0.6, s=10)
    plt.xlabel('Scan Time (minutes)')
    plt.ylabel('Spectrum ID')
    plt.title(f'Spectrum ID vs Scan Time\n{os.path.basename(mzml_file)}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Total spectra plotted: {len(spectrum_ids)}")
    print(f"Scan time range: {min(scan_times):.2f} - {max(scan_times):.2f} minutes")
    print(f"Spectrum ID range: {min(spectrum_ids)} - {max(spectrum_ids)}")

# Usage
mzml_file = "mzML/plate1_mzML/20241127_Scapis_DIA_P1A1.mzML"
plot_spectrum_vs_scantime(mzml_file)